In [1]:
import db_helpers as dbh
import db_parse as dbp

import json
import pandas as pd
from pprint import pprint
import os

In [2]:
db_name = "dos"
schema = "schema.sql"
revision = "test"
materialization = "vnet"
session = "quic_test"
data_dir = "/usr/local/dos-mitigation/data"
do_wipe = True # If True, ALL existing data will be overwritten

In [3]:
if do_wipe:
    try:
        dbh.db_drop(db_name)
    except:
        pass
    dbh.db_add(db_name, schema)

elif db_name not in dbh.db_list():
    dbh.db_add(db_name, schema)

In [4]:
conn = dbh.Connection("dos")
print("Connected to database: {}".format(conn.db_name()))

Connected to database: dos


In [5]:
revision_row = {
    "hash": "5bcc425177d073587dc0f78bf9acf084676d2061",
    "metal": True,
    "model_file": None,
    "nickname": revision
}
if do_wipe:
    conn.insert_dict_as_row('revisions', revision_row)
revision_id = conn.nickname_id("revisions", revision)

In [6]:
materialization_row = {
    "nickname": materialization,
    "revision": revision_id,
    "inventory_file": None
}
if do_wipe:
    conn.insert_dict_as_row('materializations', materialization_row)
materialization_id = conn.nickname_id("materializations", materialization)

In [7]:
print("Parsing session: {}".format(session))

with open("{}/{}/{}/.parameters.json".format(data_dir, materialization, session)) as f:
    parameters = json.load(f)

session_row = {
    "materialization": materialization_id,
    "parameters": parameters,
    "nickname": session
}
conn.insert_dict_as_row("sessions", session_row)
session_id = dbh.result_as_value(conn.db_query("select id from sessions where nickname='{}'".format(session)))

Parsing session: quic_test


In [8]:
for experiment in os.listdir("{}/{}/{}".format(data_dir, materialization, session)):
    if experiment.startswith('.'):
        continue
    dbp.parse_experiment(conn, materialization, session, experiment)
    dbp.analyze_experiment(conn, experiment)

Parsing experiment: 2023_11_08_14_10_31
Parsing experiment: 2023_11_08_14_05_11
Parsing experiment: 2023_11_08_14_15_52
Parsing experiment: 2023_11_08_13_59_51
